# SQL Workshop Driver
Run this notebook to prepare the workspace for the HLS SQL Workshop.
It will:
1. Create schemas `cms` and `ai` if they do not exist.
2. Create workshop views (fact_patient_claims, rprt_patient_claims, dim_beneficiary, dim_provider, dim_date, dim_diagnosis) in the workshop schema.
3. Optionally create the SQL warehouse and/or sample dashboard.

**Prerequisites:** Run **BuildTheWorkspace.ipynb** (in `dbx_hls_workspace_setup/`) first so that `healthcare_dev.gold` contains the CMS and dimensional gold tables (dim_cms_beneficiary, dim_cms_provider, fact_cms_outpatient_claims, fact_cms_inpatient_claims, dim_date, etc.). Use the same catalog and gold_schema as in that notebook (catalog=healthcare_dev, gold_schema=gold).

In [0]:
# Add workshop scripts to path (notebook is under .../SQL Workshop/notebooks/instructors/)
import sys
import os
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
workshop_root = notebook_path.rsplit('/', 3)[0]  # .../SQL Workshop
scripts_path = os.path.join(workshop_root, 'scripts')
if os.path.exists(scripts_path):
    sys.path.insert(0, scripts_path)
else:
    sys.path.insert(0, os.path.join(os.getcwd(), 'scripts'))

In [0]:
# Widgets
dbutils.widgets.text("catalog", "healthcare_dev", "Catalog")
dbutils.widgets.text("workshop_schema", "cms", "Workshop schema")
dbutils.widgets.text("gold_schema", "gold", "Gold schema")
dbutils.widgets.dropdown("create_warehouse", "false", ["true", "false"], "Create SQL warehouse")
dbutils.widgets.dropdown("create_views", "true", ["true", "false"], "Create workshop views")
dbutils.widgets.dropdown("create_dashboard", "false", ["true", "false"], "Create sample dashboard")

In [0]:
catalog = dbutils.widgets.get("catalog")
workshop_schema = dbutils.widgets.get("workshop_schema")
gold_schema = dbutils.widgets.get("gold_schema")
create_warehouse = dbutils.widgets.get("create_warehouse") == "true"
create_views = dbutils.widgets.get("create_views") == "true"
create_dashboard = dbutils.widgets.get("create_dashboard") == "true"
print(f"Catalog: {catalog}, Workshop schema: {workshop_schema}, Gold schema: {gold_schema}")
print(f"Create warehouse: {create_warehouse}, Create views: {create_views}, Create dashboard: {create_dashboard}")

In [0]:
# Create schemas and views
from create_workshop_views import create_workshop_schemas, create_workshop_views

create_workshop_schemas(spark, catalog, workshop_schema, "ai")
if create_views:
    create_workshop_views(spark, catalog, gold_schema, workshop_schema)
    print("Workshop views created.")
else:
    print("Skipped view creation.")

In [0]:
# Optional: Create SQL warehouse
if create_warehouse:
    from create_sql_warehouse import create_workshop_warehouse
    wh_id = create_workshop_warehouse()
    print(f"Warehouse ID: {wh_id}")
else:
    wh_id = None
    print("Skipped warehouse creation. Create SQL_Workshop_Serverless_Warehouse manually if needed.")

In [0]:
# Optional: Create sample dashboard (requires warehouse_id)
if create_dashboard:
    from databricks.sdk import WorkspaceClient
    w = WorkspaceClient()
    warehouses = list(w.warehouses.list())
    wh = next((x for x in warehouses if x.name == "SQL_Workshop_Serverless_Warehouse"), None)
    if wh:
        from create_dashboard import create_workshop_dashboard
        create_workshop_dashboard(warehouse_id=wh.id, catalog=catalog, schema=workshop_schema)
    else:
        print("SQL warehouse not found. Create dashboard manually or run with create_warehouse=true first.")
else:
    print("Skipped dashboard creation.")

In [0]:
# Summary
print("\n=== Workshop setup summary ===")
print(f"Catalog: {catalog}")
print(f"Workshop schema: {catalog}.{workshop_schema}")
print(f"Gold schema: {catalog}.{gold_schema}")
print("\nObjects for analysts:")
print(f"  - {catalog}.{workshop_schema}.fact_patient_claims")
print(f"  - {catalog}.{workshop_schema}.rprt_patient_claims")
print(f"  - {catalog}.{workshop_schema}.dim_beneficiary")
print(f"  - {catalog}.{workshop_schema}.dim_provider")
print(f"  - {catalog}.{workshop_schema}.dim_date")
print(f"  - {catalog}.{workshop_schema}.dim_diagnosis")
print("\nNext: Run VALIDATION_CHECKLIST.md and configure Genie per GENIE_SPACE_CONFIG.md.")